<a href="https://colab.research.google.com/github/otitamario/sp-pa-gep/blob/main/experiments/Example5_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clone the repository into Colab runtime
!git clone https://github.com/otitamario/sp-pa-gep.git

# Move into repo root
%cd sp-pa-gep

# Make sure Python sees the project root
import sys
sys.path.append(".")

Cloning into 'sp-pa-gep'...
remote: Enumerating objects: 282, done.
remote: Counting objects: 100% (98/98), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 282 (delta 68), reused 23 (delta 23), pack-reused 184 (from 1)
Receiving objects: 100% (282/282), 4.39 MiB | 26.30 MiB/s, done.
Resolving deltas: 100% (136/136), done.
/content/sp-pa-gep


In [2]:
import os
# =========================
# OUTPUT DIRECTORY (save plots here)
# =========================
FIGDIR = "figures"
os.makedirs(FIGDIR, exist_ok=True)

In [4]:
import os
import numpy as np

# Adjust these imports to match your repo structure
# If your repo has src/benchkit.py, then typically:
from src import benchkit as bk


def experiment_ex5_L2_unitball_positivepart(
    N_grid: int = 500,
    r: float = 0.5,
    maxit: int = 150,
    inner_tol: float = 1e-10,
    inner_maxit: int = 400,
    stop_tol_step: float = 0.0,   # fixed budget (as in your other experiments)
    seed: int = 123,
    outdir: str = "figures",
    tag: str = "ex5_L2_unitball_positivepart",
):
    """
    Experiment 5:
      H = L^2([0,1]) discretized on N_grid uniform points.
      K = closed unit ball in L^2.
      F(x) = max(0, x) pointwise.
      Resolvent: u = P_K(x - r F(u)) solved by inner fixed-point:
          z_{k+1} = P_K(x - r F(z_k))
      SPPA: x_{n+1} = alpha_n u_anchor + (1-alpha_n) u_n
      WPPA: x_{n+1} = alpha_n x_n + (1-alpha_n) u_n
      alpha_n = 1/(n+2)
    """

    # --- Grid spacing (use N_grid points on [0,1]) ---
    # If you use endpoints, spacing is 1/(N_grid-1). This is standard for uniform grid.
    h = 1.0 / (N_grid - 1)

    # --- Discrete L2 norm: ||x||_{L2} ~ sqrt(h) * ||x||_2 ---
    def norm_L2(x: np.ndarray) -> float:
      return float(np.sqrt(h) * np.linalg.norm(x))

    # Unit L2 ball corresponds to Euclidean radius = 1/sqrt(h)
    euclid_radius = 1.0 / np.sqrt(h)
    def error_fn(x: np.ndarray) -> float:
      return norm_L2(x)

    def proj_K(x: np.ndarray) -> np.ndarray:
        """Projection onto L2-unit ball (via equivalent Euclidean ball scaling)."""
        n2 = np.linalg.norm(x)
        if n2 <= euclid_radius:
            return x
        return (euclid_radius / n2) * x

    def F(x: np.ndarray) -> np.ndarray:
        return np.maximum(x, 0.0)

    # Residual: R(x) = ||x - P_K(x - r F(x))||_{L2} (discrete)
    # NOTE: We also clip to eps>0 to avoid semilogy(0) issues in plotting.
    def residual_fn(x: np.ndarray) -> float:
      y = proj_K(x - r * F(x))
      R = norm_L2(x - y)
      return float(R)
    # Inner fixed-point to compute u_n = P_K(x_n - r F(u_n))
    def resolvent_fixed_point(xn: np.ndarray) -> tuple[np.ndarray, int]:
        z = proj_K(xn)  # safe initialization in K
        for it in range(1, inner_maxit + 1):
            z_next = proj_K(xn - r * F(z))
            if norm_L2(z_next - z) <= inner_tol:
                return z_next, it
            z = z_next
        return z, inner_maxit

    # Step functions for benchkit.run:
    # IMPORTANT: do NOT put "μ=" in method_name for this experiment.
    # Also: to ensure the residual plotted is the *true* residual, we do NOT rely on benchkit's "u" shortcut.
    # We'll let benchkit call residual_fn(x_next) by not returning "residual" or "u".
    def make_sppa_step(u_anchor: np.ndarray):
      def step_fn(x: np.ndarray, k: int):
          alpha = 1.0 / (k + 2.0)
          u, inner_iters = resolvent_fixed_point(x)

          x_next = alpha * u_anchor + (1.0 - alpha) * u

          return x_next, {
              "u": u,  # allows benchkit to compute step/residual consistently if needed
              "inner_iters": inner_iters,
              "inner_hit_max": int(inner_iters >= inner_maxit),
              "ok": 1,  # fixed-point always returns something; set 0 only if you add failure detection
              "constraint_violation": 0.0,  # proj_K enforces feasibility
          }
      return step_fn
    def make_wppa_step():
      def step_fn(x: np.ndarray, k: int):
          alpha = 1.0 / (k + 2.0)
          u, inner_iters = resolvent_fixed_point(x)

          x_next = alpha * x + (1.0 - alpha) * u

          return x_next, {
              "u": u,
              "inner_iters": inner_iters,
              "inner_hit_max": int(inner_iters >= inner_maxit),
              "ok": 1,
              "constraint_violation": 0.0,
          }
      return step_fn

    # --- Initial point x0 chosen randomly in K ---
    rng = np.random.default_rng(seed)
    x0 = rng.standard_normal(N_grid)
    x0 = proj_K(x0)  # ensure x0 ∈ K

    # For SPPA, your text says "anchor u = 0" often,
    # but you can choose any fixed anchor in K. Here we use u=0 (classic Halpern anchor).
    u_anchor = np.zeros_like(x0)

    logs_sppa, sum_sppa = bk.run(
        method_name="SPPA",
        x0=x0,
        max_iter=maxit,
        stop_tol_step=stop_tol_step,
        step_fn=make_sppa_step(u_anchor=u_anchor),
        residual_fn=residual_fn,
        error_fn=error_fn,  # x* is 0, but we keep experiment focused on residual/step
    )

    logs_wppa, sum_wppa = bk.run(
        method_name="WPPA",
        x0=x0,
        max_iter=maxit,
        stop_tol_step=stop_tol_step,
        step_fn=make_wppa_step(),
        residual_fn=residual_fn,
        error_fn=error_fn,
    )

    # --- Plots and LaTeX table using benchkit utilities ---
    os.makedirs(outdir, exist_ok=True)

    outputs = bk.make_standard_plots(
        logs_by_method={"SPPA": logs_sppa, "WPPA": logs_wppa},
        outdir=outdir,
        tag=tag,
        plot_residual=True,
        plot_error=False,
        show_title=False,
    )

    tables_tex = bk.latex_tables_split(
      [sum_sppa, sum_wppa],
      caption_perf="Computational performance for Experiment 5.",
      label_perf="tab:ex5_perf",
      caption_acc=r"Accuracy metrics for Experiment 5 (discretized $L^2$ unit-ball VI).",
      label_acc="tab:ex5_acc",
    )

    print(tables_tex)
    print("Saved:", outputs)

    return outputs, (sum_sppa, sum_wppa), (logs_sppa, logs_wppa)


if __name__ == "__main__":
    experiment_ex5_L2_unitball_positivepart()

\begin{table}[!ht]
\centering
\begin{tabular}{lrrrrrrrr}
\hline
Method & It. & Tot (s) & Avg res (s) & Step$_f$ & Avg in. & Max in. & \% max & Succ. (\%)\\
\hline
SPPA & 150 & 0.0225 & 0.00012 & 7.0188e-04 & 5.55 & 33 & 0.0 & 100.0 \\
WPPA & 150 & 0.0300 & 0.00017 & 1.2024e-15 & 7.75 & 33 & 0.0 & 100.0 \\
\hline
\end{tabular}
\caption{Computational performance for Experiment 5.}
\label{tab:ex5_perf}
\end{table}

\begin{table}[!ht]
\centering
\begin{tabular}{lrrr}
\hline
Method & Constr. viol. & $\lVert x_n-\bar{x}\rVert$ & $R(x_n)$\\
\hline
SPPA & 0 & 4.7131e-03 & 7.0188e-04 \\
WPPA & 0 & 7.1167e-01 & 1.2024e-15 \\
\hline
\end{tabular}
\caption{Accuracy metrics for Experiment 5 (discretized $L^2$ unit-ball VI).}
\label{tab:ex5_acc}
\end{table}

Saved: {'step': 'figures/ex5_L2_unitball_positivepart_steps.png', 'residual': 'figures/ex5_L2_unitball_positivepart_residual.png'}
